In [9]:
!pip install -U deep_translator
!pip install transformers
!pip install git+https://github.com/PrithivirajDamodaran/Parrot.git
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-8h4bm25w
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-8h4bm25w
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#TRANSLATOR
from deep_translator import GoogleTranslator
translated = GoogleTranslator(source='auto', target='es')

In [12]:
#Check similatiries between two strings
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def similar(a, b):
  #return re.findall("\d+\.\d+", str(util.dot_score(a, b)))[0]
  first = model.encode(a)
  second = model.encode(b)
  return util.dot_score(first, second)

In [13]:
import pandas as pd
from transformers import pipeline

#Summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

#Importar Parrot (paraphraser)

from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

#Generar una semilla para la recreación de las frases
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

#Cargar el modelo parrot
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [14]:
#Read the wrapper dataframe

data = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Wrappers/Wiki.xlsx", index_col=0)

In [15]:
#Create Dataset
dat = {"Consejo" : ["prueba"], "Resumen":["year"], "Relación Semántica": ["prueba"]}
post = pd.DataFrame(dat)
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Benchmarks/SummaryBenchmark.xlsx")

In [16]:
#Extract the data and apply the models

for i in range(len(data)) :
  wik = data.loc[i, "Text"]

  #SUMMARIZATION
  resume = summarizer(wik, max_length=60, min_length=30, do_sample=False)
  
  #PARAPHRASER

  #TRADUCTOR
  trans = translated.translate(resume[0]["summary_text"])

  prueba = {"Consejo":wik, "Resumen": resume[0]["summary_text"], "Relación Semántica":similar(wik, resume[0]["summary_text"])[0][0].item()}
  post = post.append(prueba, ignore_index=True)
  #print(similar(wik, resume[0]["summary_text"])[0][0].item())

Your max_length is set to 60, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 60, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 60, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


In [17]:
 post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Benchmarks/SummaryBenchmark.xlsx")